# 11/5 Notebook - Tries

Hi everyone! This week, we'll be moving away from high-level programming to get a taste of the lower-level programming problems that NLP scientists dealt with in the past. Specifically, we'll be learning about the different ways that we can store a large list of words

Objectives:
- Learn an introduction to `object-oriented programming` and `recursion` in Python
- Briefy explore familiar data structures such as `lists`
- Create a new data structure, a `Trie`

To finish this notebook, you'll have to complete the following methods:
1. `binary_search()`
2. `insert()`
3. `find()`
4. `simulate_find()`

## Part 1: Loading the Data

Let's start by importing some libaries

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import time

We'll now load in our data set, `Colins Scrabble Words (2019)`

In [2]:
file = open("Collins Scrabble Words (2019).txt", encoding = "utf-8")
text = file.read()
scrabble_words = text.split("\n")[2:]

To verify the data, we'll print a random sample of words

In [3]:
random.choices(scrabble_words, k = 5)

['VOLATILES', 'TWITS', 'MALTOLS', 'LICHENOSE', 'BITUMINIZATIONS']

## Part 2: Lists

First, we'll compare two ways of sorting our list, if it's `unsorted` versus `sorted`

The function below performs `linear search`, assuming that the list is unsorted. It iterates through each element in the list and checks to see if the key and element in the list match

In [4]:
def linear_search(key, words):
    # makes sure the word is upper-case
    key = key.upper()
    for word in words:
        if key == word:
            return True
    return False

You'll see below that the function works as expected

In [5]:
print(linear_search("thisisnotaword", scrabble_words))
print(linear_search("eureka", scrabble_words))

False
True


Now let's do a quick analysis of how good this function is. Clearly, the `best case` scenario is when we are looking for the first item in the list, and the `worst case` is if it doesn't exist in the list

In [6]:
# best case
start_time = time.time()
linear_search(scrabble_words[0], scrabble_words)
# worst case
end_time = time.time()
linear_search("notaword", scrabble_words)
# random word
end_no_find_time = time.time()
linear_search(random.choice(scrabble_words), scrabble_words)

print(f"Best Case Time: {end_time - start_time}")
print(f"Worst Case Time: {end_no_find_time - end_time}")
print(f"Random Case Time: {time.time() - end_no_find_time}")

Best Case Time: 0.0
Worst Case Time: 0.017859458923339844
Random Case Time: 0.015401601791381836


This isn't that bad! But we can do better with `binary search`

Complete the method `binary_search()` below, which does the following:
1. Returns `False` for the edge case (`start >= end`)
2. Calculates the `middle_index`, the average of `start` and `end`
3. Recurses appropriately for `key > words[middle_index]` and `key < words[middle_index]`

In [7]:
def binary_search(start, end, words, key):
    # base case
    if (start >= end):
        return False
    # get the middle index
    middle_index = int((start + end) / 2)
    
    # different comparison cases 
    if (key == words[middle_index]):
        return True
    if (key > words[middle_index]):
        return binary_search(middle_index + 1, end, words, key)
    else:
        return binary_search(start, middle_index - 1, words, key)

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint for Step 1</b></font>
</summary>
<p>
<ul>
    <li>replace ... with <code>False</code></li>
</ul>
</p>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints for Step 2</b></font>
</summary>
<p>
<ul>
    <li>Don't forget to cast the number to an <code>int</code></li>
    <li>Set <code>middle_index</code> to <code>int((start + end) / 2)</code></li>
</ul>
</p>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints for Step 3</b></font>
</summary>
<p>
<ul>
    <li>If <code>key > words[middle_index]</code> return <code>binary_search(middle_index + 1, end, words, key)</code></li>
    <li>If <code>key < words[middle_index]</code> return <code>binary_search(start, middle_index - 1, words, key)</code></li>
</ul>
</p>

In [8]:
# run this cell to test your code
in_dict = binary_search(0, len(scrabble_words), scrabble_words, "AARDVARK")
not_in_dict = binary_search(0, len(scrabble_words), scrabble_words, "YOOPDEYOOP")

if in_dict and not not_in_dict:
    print("Actual master of searching")
else:
    print("Brush up on those algorithms homie")

Actual master of searching


Now we'll compare the two by using random sampling

In [9]:
linear_time = 0
binary_time = 0
for sample in range(1000):
    # test linear search
    linear_start = time.time()
    linear_search(random.choice(scrabble_words), scrabble_words)
    linear_time += (time.time() - linear_start)
    
    # test binary search
    binary_start = time.time()
    binary_search(0, len(scrabble_words), scrabble_words, random.choice(scrabble_words))
    binary_time += (time.time() - binary_start)

As expected, the `binary search` time is much faster

In [10]:
print(f"Average Linear Search Time {linear_time / 1000}")
print(f"Average Binary Search Time {binary_time / 1000}")

Average Linear Search Time 0.010290016889572144
Average Binary Search Time 3.950238227844238e-05


You might be wondering why we need another data structure, but I'll present a problem that is very slow to solve by using a list:

Suppose we're given a list of letters, and we want to form all of the possible words in our `Scrabble Dictionary` that can be made with these letters

Our general algorithm (which I will implement) is as follows:
   1. Create a `numpy array` of word counts for each `word` and our `key`
   2. Check if our word counts is greater for our key for every letter
   3. If it is, add it to our solutions

In [11]:
# helper method to create our word counts
def create_word_counts(w):
    w = w.upper()
    word_counts = np.zeros(26)
    for letter in w:
        word_counts[(ord(letter) - ord("A"))] += 1
    return word_counts

In [12]:
# main method
def unscramble_list(scramble, words):
    # initialize list and scrambled word counts
    possible_words = []
    scramble_count = create_word_counts(scramble.upper())
    
    # iterate through each word
    for word in words:
        # create the word count
        word_count = create_word_counts(word)
        
        # compare the numbers in the count
        if (np.all(word_count <= scramble_count)):
            possible_words.append(word)
    
    return possible_words

You can try unscrambling words below!

In [13]:
unscramble_list("ant", scrabble_words)

['AN', 'ANT', 'AT', 'NA', 'NAT', 'TA', 'TAN']

You might notice that this takes longer than it should

In [14]:
start_time = time.time()
unscramble_list("abcdefghijklmnopqrstuvwxyz", scrabble_words)
print(f"Run Time: {time.time() - start_time} sec")

Run Time: 5.897517442703247 sec


This may not seem like a long time, but if a program is doing a lot of other stuff, this will be a **huge** problem. Fortunately, another data structure, the `Trie`, will come to our rescue

## Part 3: Implementing the Trie

Recall what the `Trie` looks like:

<img src = "./Trie.png" style = "width:300px; height:auto;"></img>

For a better visualization of how inserting and searching for elements`Trie` works, you can click [here](https://www.cs.usfca.edu/~galles/visualization/Trie.html)

We'll start by defining a `TrieNode` class (each circle in the picture)

In [15]:
class TrieNode:
    def __init__(self): 
        # give it 26 children, initialized to None
        self.children = np.repeat(None, 26)
        # if the node is a leaf, it is the end of the word
        self.is_leaf = False

Here we have a simple helper function for converting a character to an integer between 0 and 26

In [16]:
def char_to_int(letter):
    return ord(letter) - ord("A")

Complete the function `insert()` below, which takes a `TrieNode`, root, and a `word` as its arguments, and performs the following steps:
1. Sets `node` to our `root`
2. Uses `char_to_int()` to get the correct value for `letter_index`
3. Sets `node.children[letter_index]` to a new `TrieNode()` if it doesn't exist as a child
4. Updates the value of `node` to the appropriate child
5. Updates `node.is_leaf` to `True`

In [17]:
def insert(root, word):
    # initialize the node
    node = root
    
    # iterate through each letter
    for letter in word:
        # calculate the letter index
        letter_index = char_to_int(letter) 
        
        # check if there is no child and create one if needed
        if (node.children[letter_index] == None):
            node.children[letter_index] = TrieNode()
            
        # update our node
        node = node.children[letter_index]
        
    # make the end node a leaf node
    node.is_leaf = True

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint for Step 1</b></font>
</summary>
<p>
<ul>
    <li>Set <code>node</code> equal to <code>root</code></li>
</ul>
</p>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints for Step 2</b></font>
</summary>
<p>
<ul>
    <li>Use<code>char_to_int()</code> with <code>letter</code> as the parameter</li>
    <li>Set <code>letter_index</code> equal to <code>char_to_int(letter)</code></li>
</ul>
</p>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints for Step 3</b></font>
</summary>
<p>
<ul>
    <li>Create a <code>TrieNode</code> by using the constructor, <code>TrieNode()</code></li>
    <li>Use the line of code: <code>node.children[letter_index] = TrieNode()</code></li>
</ul>
</p>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint for Step 4</b></font>
</summary>
<p>
<ul>
    <li>Set <code>node</code> equal to <code>node.children[letter_index]</code></li>
</ul>
</p>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints for Step 5</b></font>
</summary>
<p>
<ul>
    <li>Set <code>is_leaf</code> of <code>node</code> to <code>True</code> using the <code>.</code> operator</li>
    <li><code>node.is_leaf = True</code></li>
</ul>
</p>

Now we'll create a simple function that uses `insert()` to appropriately create our `Trie`

In [18]:
def insert_words(trie, words):
    # iterate through each word
    for word in words:
        # insert it into the trie
        insert(trie, word)

All of our methods are done, so it's time to create our `Trie`

In [19]:
scrabble_trie = TrieNode()
insert_words(scrabble_trie, scrabble_words)

Although it took a bit to create our `Trie`, it has benefits in some of its other methods. To showcase this, we'll create a function for checking if words exist in our dictionary

Complete the function `find()` which does the following:
1. Initializes `node` to the `trie`
2. Returns appropriately if the `node == None`
3. Sets the value of `letter_index` using `char_to_int()`
4. Updates `node` to the value of its child
5. Returns `True` if the `node` is not `None` and it is a leaf

In [20]:
def find(trie, word):
    # initializes the node
    node = trie
    
    # iterates through each letter in the word
    for letter in word:
        # if the node is None, the word doesn't exist
        if (node == None):
            return False
        
        # get the index of the letter
        letter_index = char_to_int(letter)
        # sets the next node
        node = node.children[letter_index]
    
    # returns true if the node doesn't equal None and is a leaf
    return node != None and node.is_leaf

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint for Step 1</b></font>
</summary>
<p>
<ul>
    <li>Set <code>node</code> equal to <code>trie</code></li>
</ul>
</p>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint for Step 2</b></font>
</summary>
<p>
<ul>
    <li>You want to return <code>False</code> in this case</li>
</ul>
</p>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints for Step 3</b></font>
</summary>
<p>
<ul>
    <li>Use<code>char_to_int()</code> with <code>letter</code> as the parameter</li>
    <li>Set <code>letter_index</code> equal to <code>char_to_int(letter)</code></li>
</ul>
</p>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint for Step 4</b></font>
</summary>
<p>
<ul>
    <li>Set <code>node</code> equal to <code>node.children[letter_index]</code></li>
</ul>
</p>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint for Step 5</b></font>
</summary>
<p>
<ul>
    <li>Return the condition <code>node != None and node.is_leaf</code></li>
</ul>
</p>

In [21]:
# run this cell to test your code for find and insert
in_trie = find(scrabble_trie, "DEARTH")
not_in_trie = find(scrabble_trie, "SPLEARTH")

if in_trie and not not_in_trie:
    print("Congratulations, sailor!")
else:
    print("Back to the seven seas with you, sailor")

Congratulations, sailor!


Mathematically, the runtime of binary search is $O(log_2(n))$, while the runtime of find for our `Trie` is $O(m)$, where $m$ is the length of our word. This is a considerable difference for very long dictionaries!

In the next cell, we'll prove this by simulation

Create a function called `simulate_find()` which does the following:
1. Gets a random word with `random.choice()`. Its documentation can be found [here](https://www.w3schools.com/python/ref_random_choice.asp)
2. Calculates `trie_runtime`
3. Increments the total trie runtime in `trie_time`
4. Calculates `binary_runtime`
5. Increments the total binary search runtime in `binary_time`
6. Calculates `average_trie_time` and `average_binary_time`

In [22]:
def simulate_find(trie, words, num_simulations):
    # initializes the variables
    trie_time = 0
    binary_time = 0
    
    # iterates for each simulation
    for sim in range(num_simulations):
        # gets the random word
        random_word = random.choice(words)
        
        # times for trie find
        trie_start = time.time()
        find(trie, random_word)
        trie_end = time.time()
        
        # adjusts trie total runtime
        trie_runtime = trie_end - trie_start
        trie_time += trie_runtime
        
        # times for binary search
        binary_start = time.time()
        binary_search(0, len(words), words, random_word)
        binary_end = time.time()
        
        # adjusts binary search total runtime
        binary_runtime = binary_end - binary_start
        binary_time += binary_runtime
    
    # calculates the average
    average_trie_time = trie_time / num_simulations
    average_binary_time = binary_time / num_simulations
    
    # return the values
    return (average_trie_time, average_binary_time)

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints for Step 1</b></font>
</summary>
<p>
<ul>
    <li>You want to use the <code>random.choice()</code> function with <code>words</code> as the parameter</li>
    <li>Use the line of code <code>random_word = random.choice(words)</code></li>
</ul>
</p>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint for Step 2</b></font>
</summary>
<p>
<ul>
    <li><code>trie_runtime</code> is the difference between <code>trie_end</code> and <code>trie_start</code></li>
</ul>
</p>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint for Step 3</b></font>
</summary>
<p>
<ul>
    <li>Increment <code>trie_time</code> by <code>trie_runtime</code></li>
</ul>
</p>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint for Step 4</b></font>
</summary>
<p>
<ul>
    <li><code>binary_runtime</code> is the difference between <code>binary_end</code> and <code>binary_start</code></li>
</ul>
</p>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint for Step 5</b></font>
</summary>
<p>
<ul>
    <li>Increment <code>binary_time</code> by <code>binary_runtime</code></li>
</ul>
</p>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint for Step 6</b></font>
</summary>
<p>
<ul>
    <li><code>average_trie_time</code> is equal to <code>trie_time</code> divided by <code>num_simulations</code></li>
    <li><code>average_binary_time</code> is equal to <code>binary_time</code> divided by <code>num_simulations</code></li>
</ul>
</p>

You should notice that on average, the `Trie` bests `binary_search()` by a very small margin

In [23]:
runtimes = simulate_find(scrabble_trie, scrabble_words, 100000)
print(f"Average Trie Find Runtime: {runtimes[0]}")
print(f"Average Binary Search Runtime: {runtimes[1]}")

Average Trie Find Runtime: 1.1225299835205079e-05
Average Binary Search Runtime: 1.9129645824432374e-05


The performance is pretty similar between our two methods for `find()`, but now let's revisit our `unscramble()` method, which I have coded for you again 😎

In [24]:
def unscramble(node, scramble, possible_words, word):
    # if it's a leaf, add it to the list
    if node.is_leaf:
        possible_words.append(word)
    
    # return early if there are no more letters left
    if (np.sum(scramble) == 0):
        return
    
    # go through each letter
    for check_index in range(26):
        # condition for if the word is viable
        if (scramble[check_index] > 0 and node.children[check_index] != None):
            
            # adjust variables to recurse
            word += chr(check_index + ord("A"))
            scramble[check_index] -= 1
            
            # recursively call
            unscramble(node.children[check_index], scramble, possible_words, word)
            
            # reset to backtrack
            scramble[check_index] += 1
            word = word[:-1]

Take a look at how much faster this is!

In [25]:
possible_words = []
scramble_count = create_word_counts("ant")
unscramble(scrabble_trie, scramble_count, possible_words, "")
possible_words

['AN', 'ANT', 'AT', 'NA', 'NAT', 'TA', 'TAN']

Let's do some runtime analysis

In [26]:
# function for simulating
def simulate_scramble(trie, words, num_simulations):
    
    # initialize variables
    trie_time = 0
    list_time = 0
    
    # iterate through each simulation
    for sim in range(num_simulations):
        # random word
        random_word = random.choice(words)
        
        # calculate trie runtime
        trie_start = time.time()
        unscramble(trie, create_word_counts(random_word), [], "")
        trie_time += (time.time() - trie_start)
        
        # calculate list runtime
        list_start = time.time()
        unscramble_list(random_word, words)
        list_time += (time.time() - list_start)
        
    # return averages
    return (trie_time / num_simulations, list_time / num_simulations)

Compared to `find()`, it is **much** more obvious that `scramble()` is better with the `Trie` implementation compare to the `List` implementation

*Note: This cell takes ~20-30 seconds to run*

In [27]:
trie_scramble, list_scramble = simulate_scramble(scrabble_trie, scrabble_words, 10)

print(f"Average Trie Scramble Runtime: {trie_scramble}")
print(f"Average List Scramble Runtime: {list_scramble}")

Average Trie Scramble Runtime: 0.029515576362609864
Average List Scramble Runtime: 6.268676829338074
